We download the dataset directly from *sklearn.datasets*. No need to have a local copy of the datasets

In [1]:
from sklearn.datasets import fetch_mldata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

mnist = fetch_mldata("MNIST original")
X = mnist.data/255.0  # Dividing by 255.0 is to normalize the data between 0 and 1
y = mnist.target      # Labels are stored as a vector 
print(X.shape)
print(y.shape)

(70000, 784)
(70000,)


Convert the above matrix $X$ and vector $y$ into a dataframe so that life becomes easier ;)

In [2]:
# Naming the features as 'pixel #' for all the columns in the matrix X
features = ['pixel '+str(i) for i in range(X.shape[1])]  
df = pd.DataFrame(X, columns=features) # Setting the array features as columns for the dataframe 
df['label'] = y
#df['label'] = df['label'].apply(lambda i: str(i))

From the above 70000 data points, we don't want to use all the 70000 data points so we sample some data points at random. The randomization is important as the dataset is sorted by its label. (i.e., the first 7000 data points are labelled 0's, next 7000 are 1's, etc.). To ensure randomization we'll create a random permutation of the number 0 to 69999 which allows us later to select the first 5000 or 10,000 data points for our calculations and visualization. 

In [3]:
#random_permutation contains the indices from 0 through 69,999 randomly
random_permutation = np.random.permutation(df.shape[0]) 

Now we have our dataframe and our randomization vector. Let's look at the actual digits obtained after randomization. We shall generate 30 plots of these randomly selected images. 

In [4]:
#%matplotlib inline
#import matplotlib.pyplot as plt

# Plot the graph
#plt.gray()
#fig = plt.figure(figsize=(16,7))
#for i in range(0, 1):
#    ax = fig.add_subplot(3, 10, i+1, title='Digit: ' +
#                        str(df.loc[random_permutation[i],'label']))
#ax.matshow(df.loc[random_permutation[i],features].values.reshape((28,28)).astype(float))

Now that we have all the datapoints and the random permutation of datapoints, we now try to visualize the dataset. Since the data has $(28\times 28=784)$ features, we cannot visualize such a high dimensional data. So, we reduce the dimensions using a technique called **Principal Component Analysis**. 

We are not doing any classification here. So, there is no need to partition the dataset into train data, test data and cross-validation data. We are considering randomized datapoints just to mix all the datapoints and to avoid looking at only few thousands of datapoints of the same label. In this experiment, we will just be visualizing the high dimensional data by reducing the dimensions. Let's say we will be considering 15,000 data points (the first 15,000 indices in random_permutaion array) for our experiment. 

In [4]:
# Saving the first 15,000 datapoints into a DataFrame A.
A = pd.DataFrame(X[random_permutation[:15000]])
labels = pd.DataFrame(y[random_permutation[:15000]])
print("The indices in random_permutation: ", random_permutation[:15000])
print("Shape of matrix A:", A.shape)
print("Shape of labels y:", labels.shape)

The indices in random_permutation:  [64529 32680 62449 ..., 10294 26022 12115]
Shape of matrix A: (15000, 784)
Shape of labels y: (15000, 1)


Now we got the matrix $A$, it is to be mean centered. We find the mean $\mu_i$ for each data point in $A$ and then we transform the matrix A so that it is mean-centered. 

In [5]:
# Mean along 0 (vertical) axis: return mean for each colum. 
# Mean along 1 (horizontal) axis: return mean for each row. 
# mean_A is of size 15000 X 1
mean_A = A.mean(axis=1)

#print(mean_A.shape)
#for row in range(A.shape[0]): # for each row in A
#    for col in range(A.shape[1]): # for each column in A
#        A[row][col] -= mean_A[row]
#        print(A[row][col])


# Cloning the column for 784 times to get a matrix of size 15000 X 784
# This enables us to find the difference of matrix A and mean matrix easy. 
# Else we need to have 2 for loops which costs O(n^2) time. 

mean_A_matrix = np.tile(mean_A.transpose(), (784, 1)) 
mean_A_matrix = mean_A_matrix.transpose()
# mean_A_matrix is of size 15000 X 784

mean_centered_A = A-mean_A_matrix

Now we got the mean centered data matrix of size $15000\times 784$, we need to find the covariance matrix of mean_centered_A. And then the eigen values and eigen vectors of the covariance matrix. 

In [6]:
S = np.matmul(mean_centered_A.transpose(),mean_centered_A)

In [7]:
print(S[:1][:1])
print(S.shape)

[[ 284.65409575  284.65409575  284.65409575  284.65409575  284.65409575
   284.65409575  284.65409575  284.65409575  284.65409575  284.65409575
   284.65409575  284.65409575  284.65409575  284.65409575  284.65409575
   284.65409575  284.65409575  284.65409575  284.65409575  284.65409575
   284.65409575  284.65409575  284.65409575  284.65409575  284.65409575
   284.65409575  284.65409575  284.65409575  284.65409575  284.65409575
   284.65409575  284.65409575  284.65409575  284.64935299  284.50817795
   284.35485266  284.13648206  284.09174271  283.93290102  283.62392755
   283.43280055  283.21501934  283.05069747  283.42399134  283.54678639
   283.35526987  283.94104629  284.31345602  284.26142711  284.48868306
   284.56731767  284.64702145  284.65409575  284.65409575  284.65409575
   284.65409575  284.65409575  284.65409575  284.65409575  284.65409575
   284.4800944   284.57973591  284.09039154  283.31194022  282.11225921
   279.86020443  277.07789282  271.97648557  266.61688849  262.1

Now S is the covariance matrix, we have to find the eigen values and the corresponding eigen vectors. Then pick the highest two eigen values and the corresponding eigen vectors which correspond to the principal components 1 and 2.

In [8]:
from scipy import linalg as LA
e_vals, e_vecs = LA.eig(S)


In [11]:
print(e_vals)
print(e_vecs.shape)

[  3.54997717e+005 +0.00000000e+00j   6.45611074e+004 +0.00000000e+00j
   5.66829323e+004 +0.00000000e+00j   4.83570752e+004 +0.00000000e+00j
   4.15502859e+004 +0.00000000e+00j   3.33067625e+004 +0.00000000e+00j
   2.88014222e+004 +0.00000000e+00j   2.28221490e+004 +0.00000000e+00j
   2.24276743e+004 +0.00000000e+00j   1.85862425e+004 +0.00000000e+00j
   1.66256423e+004 +0.00000000e+00j   1.62258530e+004 +0.00000000e+00j
   1.36244564e+004 +0.00000000e+00j   1.33476763e+004 +0.00000000e+00j
   1.23027361e+004 +0.00000000e+00j   1.16112303e+004 +0.00000000e+00j
   1.08681333e+004 +0.00000000e+00j   1.04363236e+004 +0.00000000e+00j
   9.55786589e+003 +0.00000000e+00j   9.13688542e+003 +0.00000000e+00j
   8.35589567e+003 +0.00000000e+00j   7.93858063e+003 +0.00000000e+00j
   7.52770782e+003 +0.00000000e+00j   7.23735175e+003 +0.00000000e+00j
   7.00921910e+003 +0.00000000e+00j   6.64551703e+003 +0.00000000e+00j
   6.33506663e+003 +0.00000000e+00j   6.24062725e+003 +0.00000000e+00j
   5.7